In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import json

# Notebook for video processing using mediapipe

## Prototype for a single video

In [ ]:
#Mediapipe pose functions initialization
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [ ]:
#Load a video using cv2

video_input = cv2.VideoCapture('testvideo/v_JumpingJack_g01_c01.avi')

#Store the width and height for the video

frame_width = int(video_input.get(3))
frame_height = int(video_input.get(4))

In [ ]:
#Extract parameters for the annotated video generation 
viddir, inputflnm = 'testvideo/', 'v_JumpingJack_g01_c01.avi'
inflnm, inflext = inputflnm.split('.')
ex_name =  inflnm.split('_')[1]

!mkdir $inflnm

outdir = inflnm + '/'

In [ ]:
#Graphic Results file names
annotated_filename = f'{outdir}{inflnm}_annotated.{inflext}'
black_filename = f'{outdir}{inflnm}_black.{inflext}'

out_annotated = cv2.VideoWriter(annotated_filename, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
out_black = cv2.VideoWriter(black_filename, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

In [ ]:
list_land = []
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    i = 0
    while video_input.isOpened():
        ret, image = video_input.read()
        if not ret:
            break
        
        #Create black background
        background = np.zeros((frame_height, frame_width, 3), np.uint8)
            
            
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        pose_landmarks = results.pose_landmarks
        
        pose_landmarks = np.array([[lmk.x * frame_width, lmk.y * frame_height, lmk.z * frame_width] for lmk in pose_landmarks.landmark], dtype=np.float32)
        
        list_land.append(pose_landmarks)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        mp_drawing.draw_landmarks(background, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        out_annotated.write(image)
        out_black.write(background)
        
        frame_filename = f'{outdir}{inflnm}_frame{i}.jpg'
        black_filename = f'{outdir}{inflnm}_black{i}.jpg'
        
        i += 1
        
        cv2.imwrite(black_filename, background)
        cv2.imwrite(frame_filename, image)
        
video_input.release()
out_annotated.release()
out_black.release()

## Generalization

In [2]:
#Mediapipe pose functions initialization
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [3]:
#Create directories for storing the results
!mkdir annotated_videos
!mkdir black_videos
!mkdir annotated_frames
!mkdir black_frames
!mkdir jsons

In [4]:
#Iterate through the video files in directory, variables for the different outputs

directory = 'testvideo/'
ouput_annotated_videos = 'annotated_videos/'
output_black_videos = 'black_videos/'
output_annotated_frames = 'annotated_frames/'
output_black_frames = 'black_frames/'

#Initialize a dictionary to store landmark results for each frame
landmark_dict = {}

for filename in os.listdir(directory):
    
    if filename.startswith('.') == False: 
    
        current_filepath = f'{directory}{filename}'

        #Load a video using cv2
        video_input = cv2.VideoCapture(current_filepath)

        #Store the width and height for the current video
        frame_width = int(video_input.get(3))
        frame_height = int(video_input.get(4))

        #Extract parameters for the data extraction
        inflnm, inflext = filename.split('.')
        exercise_name =  inflnm.split('_')[1]

        #Current video filepath
        current_filepath = f'{directory}/{filename}'

        #Load a video using cv2
        video_input = cv2.VideoCapture(current_filepath)

        #Store the width and height for the current video
        frame_width = int(video_input.get(3))
        frame_height = int(video_input.get(4))

        #Extract parameters for the data extraction
        inflnm, inflext = filename.split('.')
        ex_name =  inflnm.split('_')[1]

        #Video outputs filepaths
        annotated_filename = f'{ouput_annotated_videos}{inflnm}_annotated.{inflext}'
        black_filename = f'{output_black_videos}{inflnm}_black.{inflext}'

        #Initializing output videos
        out_annotated = cv2.VideoWriter(annotated_filename, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
        out_black = cv2.VideoWriter(black_filename, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

            i = 0

            #Iterates through each frame in the current video
            while video_input.isOpened():
                
                ret, image = video_input.read()
                if not ret:
                    break

                #Create black background
                background = np.zeros((frame_height, frame_width, 3), np.uint8)

                #Create an image of the current frmae
                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                #Stores the landmarks of the current frame
                pose_landmarks = results.pose_landmarks
                
                frame_dict = {}
                j = 0
                for lmk in pose_landmarks.landmark:
                    frame_dict[j] = [lmk.x * frame_width, lmk.y * frame_height, lmk.z * frame_width]
                    j += 1
                
                landmark_dict[i] = frame_dict
                
                # Draws the landmarks in the two output videos
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                mp_drawing.draw_landmarks(background, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

                out_annotated.write(image)
                out_black.write(background)

                frame_filename = f'{output_annotated_frames}{inflnm}_frame{i}.jpg'
                black_frame_filename = f'{output_black_frames}{inflnm}_black{i}.jpg'

                i += 1

                cv2.imwrite(black_frame_filename, background)
                cv2.imwrite(frame_filename, image)

        video_input.release()
        out_annotated.release()
        out_black.release()
        
        #Creates json file with video landmarks
        with open(f'jsons/{inflnm}.json', 'w') as fp:
            json.dump(landmark_dict, fp)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
